In [1]:
import os; 
import pandas as pd; pd.set_option('mode.chained_assignment',None)
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import re
import xlrd
import statistics as stat
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample
from pandas.plotting import register_matplotlib_converters
from src.data_models.SmartsCsvDataModel import SmartsCsvDataModel
from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty, wtf
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame
from src.data_models.SmartsDataModel import SmartsDataModel
from src.data_models.UPMhelper import UPMalerts

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
eval_date = datetime.date(2019, 6, 30)
begin_date = datetime.date(2019, 6, 1)

trading_pairsoi = ['BTCUSD', 'LTCUSD', 'ETHUSD', 'BCHUSD', 'ZECUSD']
worst = [22440, 811415]
worsttier2 = [22440, 811415, 772544, 10078, 898579]

In [4]:
data_model = SmartsCsvDataModel(download_data=True).evaluate(instrument=trading_pairsoi, evaluation_date=eval_date, begin_date=begin_date, use_db=False)

INFO:root:Constructing Singleton
INFO:root:Using global configuration /Users/sqian/msv.yaml.
INFO:root:Completed loading configuration.


download_smarts_data 0:01:50.403391
read_csv_to_dk_dataframe 0:00:03.861241
pre_cleaning 0:08:40.346030
pre_cleaning 0:45:16.724129
pre_cleaning 0:04:50.325328
pre_cleaning 0:03:47.627392
pre_cleaning 0:02:38.438811


In [39]:
data_model.keys()

dict_keys(['BTCUSD', 'LTCUSD', 'ETHUSD', 'BCHUSD', 'ZECUSD'])

In [50]:
os.chdir('/Users/sqian/Documents/{}_output'.format(begin_date.strftime("%Y%m")))

writer = pd.ExcelWriter('{}_Wash_OrderBook.xlsx'.format(begin_date.strftime("%Y%b")))

for j in worsttier2: 
    empty = pd.DataFrame()

    for i in trading_pairsoi:
        datatemp = data_model[i]

        event_typeBOOL = datatemp['event_type'] == 'Place'
        account_idBOOL = datatemp['account_id'] == j

        datatemp2 = datatemp.loc[event_typeBOOL & account_idBOOL].groupby(['symbol','side','execution_options','order_type']).count()['event_time']
        empty = pd.concat([empty,datatemp2.reset_index()],axis=0,sort=False)
        
    finaltable = pd.pivot_table(empty,index=['symbol','order_type','execution_options'], columns='side',values='event_time').fillna(0).astype(int)
    finaltable2 = pd.concat([finaltable,finaltable.sum(axis=1)], axis=1).sort_values(0, ascending=False).rename(columns={0:'total'})
    
    finaltable2.to_excel(writer, 'fig_X.8_' + str(j))

writer.save()